**Load Libraries**

In [3]:
import os
from typing import Any, Optional, Tuple, NoReturn

import sklearn
import tensorflow as tf
import keras_tuner as kt
import kerastuner_tensorboard_logger as kt_logger 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**Paths**

In [45]:
BASE_PATH = "../"
MONITORING = os.path.join(BASE_PATH, 'logs')
DATA = os.path.join(BASE_PATH, 'data')

In [ ]:
TENSORBOARD_LOG_DIR = os.path.join(MONITORING, "tensorboard_logs")
CSV_LOG_DIR = os.path.join(MONITORING, "csv_logs")

In [46]:
TUNERS = os.path.join(DATA, "tuners")
MODELS = os.path.join(DATA, "models")

**GPU/TPU Multithreading Setup**

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)

    strategy = tf.distribute.experimental.TPUStrategy
except ValueError:
    strategy = tf.distribute.get_strategy()
    print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
except ValueError:
    tpu = None
    gpus = tf.config.experimental.list_logical_devices("GPU")

In [ ]:
if tpu:
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu, )
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
    strategy = tf.distribute.MultiWorkerMirroredStrategy([gpu.name for gpu in gpus])
    print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy()
    print('Running on single GPU ', gpus[0].name)
else:
    strategy = tf.distribute.get_strategy()
    print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

**Hyperparameters**

In [ ]:
# Fix
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# Adjustable
BATCH_SIZE = 32  # Big batch size, small learning rate
HEIGHT, WIDTH = 224, 224
IMG_SIZE = (HEIGHT, WIDTH)
IMG_FORMAT = (HEIGHT, WIDTH, 3)
EPOCHS = 100
TRIALS = 100
SEED = 42

**Load Dataset**

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'data/dataset',
    validation_split=0.2,
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

In [ ]:
NUM_CLASSES = len(dataset.class_names)

**Preprocessing**

In [ ]:
train_dataset, val_dataset = sklearn.model_selection.train_test_split(dataset, test_size=0.2)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1. / 255)

In [ ]:
normalized_ds = train_dataset.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

In [ ]:
train_ds = train_dataset.cache().shuffle().refetch(buffer_size=AUTOTUNE)
val_ds = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

**CNN**

In [ ]:
def cnn(hp: kt.HyperParameters) -> tf.keras.Sequential:
    inputs = tf.keras.Input(shape=IMG_FORMAT, dtype=tf.float32)

    # Conv & pooling tf.keras.layers
    for i in range(num_layers := hp.Int('num_layers', min_value=0, max_value=3, step=1)):
        for _ in range(2):
            filters = hp.Int(f'filters_{i}',
                             min_value=np.power(2, num_layers + i),
                             max_value=np.power(2, (num_layers + 2) + i),
                             step=8)
            x = tf.keras.layers.Conv2D(filters=filters, kernel_size=(3, 3), activation='tanh', padding='same')(x)
        x = tf.keras.layers.MaxPool2D()(x)

    # Fully connected tf.keras.layers
    x = tf.keras.layers.Flatten()(x)
    for i in range(num_layers):
        x = tf.keras.layers.Dense(units=hp.Int(f'units_{i}',
                                               min_value=np.power(2, (num_layers * 2) - i),
                                               max_value=np.power(2, np.power(2, num_layers) - i),
                                               step=8),
                                  activation='relu')(x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"])
    return model

**RNN**

In [ ]:
def rnn(hp: kt.HyperParameters) -> tf.keras.Sequential:
    inputs = tf.keras.Input(shape=IMG_FORMAT, dtype=tf.float32)
    for i in range(hp.Int('num_layers', min_value=0, max_value=3, step=1)):
        x = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(
                hp.Int(f'units_{str(i)}', min_value=8, max_value=64, step=8),
                return_sequences=True,
            )
        )(x)
    x = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(hp.Int('lstm_units', min_value=8, max_value=64, step=8),
                             return_sequences=False))(x)
    x = tf.keras.layers.Dense(hp.Int('dense_units', min_value=8, max_value=64, step=8),
                              activation='relu')(x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
    model = tf.keras.Model(inputs, x)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"])
    return model

**Utilitary For Monitoring**

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [37]:
def path_exists(path: str) -> str:
    if os.path.exists(path):
        if path[-1].isdigit():
            suffix = path[:path.rfind('_')]
            digits = int(path[path.rfind('_') + 1:])
            path = f"{suffix}_{digits + 1}"
        else:
            path = f"{path}_0"
    return path

In [ ]:
def tensorboard_logs(model_name: str) -> tf.keras.callbacks.TensorBoard:
    path = f"{globals()[model_name.upper()]}" \
           f"_BS_{BATCH_SIZE}" \
           f"_LR_{SEED}" \
           f"_EPOCHS_{EPOCHS}" \
           f"_TRIALS_{TRIALS}"
    return tf.keras.callbacks.TensorBoard(path_exists(path))

In [ ]:
def epochs_logs(model_name: str) -> tf.keras.callbacks.CSVLogger:
    path = f"{globals()[model_name.upper()]}" \
           f"_BS_{BATCH_SIZE}" \
           f"_LR_{SEED}" \
           f"_EPOCHS_{EPOCHS}" \
           f"_TRIALS_{TRIALS}"
    return tf.keras.callbacks.CSVLogger(f"{path_exists(path)}.csv")

**Training**

In [ ]:
def training(model: Any) -> NoReturn:
    model_name = model.__name__
    with strategy.scope():
        tuner = tuner = kt.BayesianOptimization(model,
                                        objective=kt.Objective('val_accuracy', direction='max'),
                                        max_trials=TRIALS,
                                        overwrite=True,
                                        project_name=path_exists(f"{TUNERS}\\{model_name}_tuner"),
                                        directory=path_exists("{TUNERS}_{model_name}"))

        # Search for best hyperparameters
        tuner.search(train_dataset,
                     epochs=EPOCHS,
                     validation_data=val_dataset,
                     callbacks=[stop_early,
                                epochs_logs(model_name),
                                tensorboard_logs(model_name)])
        # Get the optimal hyperparameters
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        print(best_hps)

        # Build model with optimal hyperparameters
        model = tuner.hypermodel.build(best_hps)
        history = model.fit(train_dataset,
                            epochs=EPOCHS,
                            validation_data=val_dataset,
                            callbacks=[stop_early,
                                       epochs_logs(model_name),
                                       tensorboard_logs(model_name)])
        val_acc_per_epoch = history.history['val_accuracy']
        best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
        print(f"best_epoch : {best_epoch}")

        hypermodel = tuner.hypermodel.build(best_hps)
        # Retrain the model with epoch with highest val_accuracy value
        hypermodel.fit(train_dataset,
                       epochs=best_epoch,
                       validation_data=val_dataset,
                       callbacks=[stop_early,
                                  epochs_logs(model_name),
                                  tensorboard_logs(model_name)])

        eval_result = hypermodel.evaluate(val_dataset)

        hypermodel.save(f"{MODELS}\\"
                        f"{model_name}"
                        f"_loss_{eval_result[0]}"
                        f"_acc_{eval_result[1]}"
                        f"_best_epoch_{best_epoch}")

In [ ]:
training(cnn)

In [ ]:
training(rnn)

**Model Evaluation**

In [47]:
models = [f'{root}\\{dir}' for root, dirs, files in os.walk(MODELS) for dir in dirs if "acc" in dir]
sort_models_per_acc = sorted(models,
                             key=lambda x: float(x[x.find('_acc_') + 5:x.find('_best_') if 'best' in x else x.find(
                                 '_para_') if "_para_" in x else None]),
                             reverse=True)

In [ ]:
best_model = tf.keras.models.load_model(sort_models_per_acc[0])

In [ ]:
predictions = best_model.predict(val_dataset).argmax(axis=1)